## Introduction

- Recursion and iteraion are equivalent.
- Iteration are just a special case of defunctionalization
- Recursion vs iteration are the same as Defunctionalization vs Refunctionalization
- #TLDR Defunctionalization v.s. Refunctionalization

## Defunctionalization

Replacing functions with data structures.

Why we need this? You cannot send a function

- Given `filter :: (a -> bool) -> [a] -> [a]`
- Target: `defunc_filter :: Filter -> [a] -> [a]`.

In [8]:
type filter =
  | IsOdd
  | IsEven
  | Lessthan of int

(* refunctionalize *)
let apply = function
  | IsOdd -> fun x -> x mod 2 <> 0
  | IsEven -> fun x -> x mod 2 = 0
  | Lessthan y -> fun x -> x < y


type filter = IsOdd | IsEven | Lessthan of int


val apply : filter -> int -> bool = <fun>


the data type can be recursive to introduce composition


In [10]:
type filter =
  | IsOdd
  | IsEven
  | Lessthan of int
  | And of filter * filter

type filter = IsOdd | IsEven | Lessthan of int | And of filter * filter


## Difference

- Higher order function (refunctionalized form) are open
- Defunctionalized form are close

In [36]:
type tree =
  | Leaf 
  | Node of tree * string * tree

let example_tree =
  let four = Node (Leaf, "4", Leaf) in
  let five = Node (Leaf, "5", Leaf) in
  let three = Node (Leaf, "3", Leaf) in
  let two = Node (four, "2", five) in
  let one = Node (two, "1", three) in
  one
 
let rec print_tree = function
  | Leaf -> ()
  | Node (left, content, right) -> print_tree left; print_endline content; print_tree right


type tree = Leaf | Node of tree * string * tree


val example_tree : tree =
  Node (Node (Node (Leaf, "4", Leaf), "2", Node (Leaf, "5", Leaf)), "1",
   Node (Leaf, "3", Leaf))


val print_tree : tree -> unit = <fun>


Step 1. cps conversion the function

In [38]:
let rec print_tree_cps tree cont =
  match tree with
  | Leaf -> cont ()
  | Node (left, content, right) -> 
    print_tree_cps left 
      (fun _ -> print_endline content; print_tree_cps right cont)

let () = print_tree_cps example_tree (Fun.id)

val print_tree_cps : tree -> (unit -> 'a) -> 'a = <fun>


4
2
5
1
3


Step 2. Defunctionalize the Continuation (cont)
Two types of continuation that could be passed in
1. base case (Done)
2. left tree continuation. Free variable analysis
   1. content of tree
   2. the right branch of the tree
   3. the continuation passed

In [39]:
(* this looks like a stack *)
type cont = 
  | Done
  | Next of (tree * string * cont)

let rec print_tree_defunc tree cont = 
  match tree with
  | Leaf -> apply cont
  | Node (left, content, right) -> print_tree_defunc left (Next (right, content, cont))
and apply = function
  | Done -> ()
  | Next (right_tree, content, kont) -> print_endline content; print_tree_defunc right_tree kont

let () = print_tree_defunc example_tree Done

type cont = Done | Next of (tree * string * cont)


val print_tree_defunc : tree -> cont -> unit = <fun>
val apply : cont -> unit = <fun>


4
2
5
1
3


Step 3 inlining the `apply`

In [40]:
let rec print_tree_defunc tree cont = 
  match tree with
  | Node (left, content, right) -> print_tree_defunc left (Next (right, content, cont))
  | Leaf -> 
    match cont with
    | Done -> ()
    | Next (right_tree, content, next) ->
      print_endline content;
      print_tree_defunc right_tree next

let () = print_tree_defunc example_tree Done


val print_tree_defunc : tree -> cont -> unit = <fun>


4
2
5
1
3


Step 4. The result after inlining give us a tail-call recursive function. We can then do the tail call elimination.

- How to do tail call elimination?
  - wrapped the code in a while true loop
  - introduce states
  - replace every occurence of tail call with state update

In [53]:
let print_tree_iter tree cont = 
  let quit_loop = ref false in
  let cont = ref cont in
  let tree = ref tree in
  while not !quit_loop do
  match !tree with
  | Node (left, content, right) -> 
    tree := left;
    cont := Next (right, content, !cont)
  | Leaf -> 
    match !cont with
    | Done -> quit_loop := true
    | Next (rtree, content, next) ->
      print_endline content;
      tree := rtree;
      cont := next
  done

let () = print_tree_iter example_tree Done

val print_tree_iter : tree -> cont -> unit = <fun>


4
2
5
1
3


1. Defunctionalization Introduced (for compilers)
2. Defunctionalization as Program Manipulation
3. Formalized Refunctionalization
   - state machine algorithm turn into direct style recursive algorithm
4. Delimited Continuation in Operating Systems

## Practical examples
- Web actions
- event loop

- Non-Blocking I/O via Notification
  - drawbacks: concurrency...
  - goal: how to make it just like the direct style but run with async
- Solution: defunctionalize
- Notification action = delimited continuation